<a href="https://colab.research.google.com/github/bholeneha/Text_Classification/blob/Fatima/NN_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings; warnings.simplefilter('ignore')

# New Section

In [1]:
import numpy as np 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import regex
#from wordcloud import WordCloud
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
from string import punctuation
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
!unzip sample_data/Data.zip
# upload the data

Streaming output truncated to the last 5000 lines.
  inflating: Data/Politics/53371.txt  
  inflating: __MACOSX/Data/Politics/._53371.txt  
  inflating: Data/Politics/54378.txt  
  inflating: __MACOSX/Data/Politics/._54378.txt  
  inflating: Data/Politics/55066.txt  
  inflating: __MACOSX/Data/Politics/._55066.txt  
  inflating: Data/Politics/178802.txt  
  inflating: __MACOSX/Data/Politics/._178802.txt  
  inflating: Data/Politics/76330.txt  
  inflating: __MACOSX/Data/Politics/._76330.txt  
  inflating: Data/Politics/178355.txt  
  inflating: __MACOSX/Data/Politics/._178355.txt  
  inflating: Data/Politics/54191.txt  
  inflating: __MACOSX/Data/Politics/._54191.txt  
  inflating: Data/Politics/178433.txt  
  inflating: __MACOSX/Data/Politics/._178433.txt  
  inflating: Data/Politics/76456.txt  
  inflating: __MACOSX/Data/Politics/._76456.txt  
  inflating: Data/Politics/76442.txt  
  inflating: __MACOSX/Data/Politics/._76442.txt  
  inflating: Data/Politics/54185.txt  
  inflating: _

In [3]:
names = []
base = 'sample_data/Data/'
with os.scandir(base) as entries:
    for entry in entries:
        if(entry.is_file() == False):
            names.append(entry.name)

try:
  names.remove(".ipynb_checkpoints")
except:
  pass
names

['Entertainment', 'Crime', 'Science', 'Politics']

In [4]:
files = {}
unique = []
for name in names:
    path = base + name+'/'
    x = []
    with os.scandir(path) as entries:
        for entry in entries:
            if(entry.is_file()):
                x.append(entry.name)
    files[name] = x
    files[name].sort()

In [5]:
for k, v in files.items():
    print(k, len(v))

Entertainment 1053
Crime 1100
Science 4000
Politics 3001


In [6]:
data_LDA = {}
i = 0

for genre in files.keys() :
    texts = files[genre]
    for text in texts:
        if text in files[genre]:
            path = base + genre + '/' + text
            with open(path, "r", encoding = "latin1") as file:
                data_LDA[i] = file.readlines()
                i = i+1
            data_LDA[i-1] = [" ".join(data_LDA[i-1]), genre] 

data_LDA = pd.DataFrame(data_LDA).T
print(data_LDA.shape)
data_LDA.columns = ['Text', 'Class']
data_LDA.head()

(9154, 2)


,Text,Class
0,"\n Archive-name: ripem/faq\n Last-update: Sun,...",Entertainment
1,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment
2,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment
3,Message-ID: <1ppvai$l79@bilbo.suite.com>\n Rep...,Entertainment
4,\n Some sick part of me really liked that p...,Entertainment


In [7]:
data_LDA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9154 entries, 0 to 9153
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9154 non-null   object
 1   Class   9154 non-null   object
dtypes: object(2)
memory usage: 214.5+ KB


In [8]:
unique = list(data_LDA.Text.unique())
len(unique)

6847

In [9]:
dic = dict(data_LDA)

In [10]:
uni = {}
i = 0
for k in range(len(list(dic['Text']))):
    if dic['Text'][k] in unique:
        uni[i] = [dic['Text'][k], dic['Class'][k]]
        unique.remove(dic['Text'][k])
        i += 1

In [11]:
data_LDA = pd.DataFrame(uni).T
print(data_LDA.shape)
data_LDA.columns = ['Text', 'Class']
data_LDA.head()

(6847, 2)


,Text,Class
0,"\n Archive-name: ripem/faq\n Last-update: Sun,...",Entertainment
1,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment
2,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment
3,Message-ID: <1ppvai$l79@bilbo.suite.com>\n Rep...,Entertainment
4,\n Some sick part of me really liked that p...,Entertainment


In [12]:
data_LDA

,Text,Class
0,"\n Archive-name: ripem/faq\n Last-update: Sun,...",Entertainment
1,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment
2,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment
3,Message-ID: <1ppvai$l79@bilbo.suite.com>\n Rep...,Entertainment
4,\n Some sick part of me really liked that p...,Entertainment
...,...,...
6842,\n In article <1th4mg$53f@usenet.INS.CWRU.Edu>...,Politics
6843,\n In article <1993May24.165636.29928@cirrus.c...,Politics
6844,Distribution: world\n Message-ID: <OAF.93May25...,Politics
6845,Message-ID: <OAF.93May25095940@klosters.ai.mit...,Politics


In [13]:
def make_wordcloud(words,title):
    cloud = WordCloud(width=1920, height=1080,max_font_size=200, max_words=300, background_color="white").generate(words)
    plt.figure(figsize=(20,20))
    plt.imshow(cloud, interpolation="gaussian")
    plt.axis("off") 
    plt.title(title, fontsize=60)
    plt.show()

In [14]:
wordnet_lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

for punct in punctuation:
    stop.append(punct)

def filter_text(text, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(text.lower())
    filtered_text = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha() and len(w) > 3]
    filtered_text = [wordnet_lemmatizer.lemmatize(w, pos="v") for w in filtered_text if not w in stop_words] 
    return " ".join(filtered_text)

In [15]:
data_LDA["filtered_text"] = data_LDA.Text.apply(lambda x : filter_text(x, stop)) 
data_LDA.head()

,Text,Class,filtered_text
0,"\n Archive-name: ripem/faq\n Last-update: Sun,...",Entertainment,archive name ripem last update post still rath...
1,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment,approve news answer request content type text ...
2,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment,approve news answer request content type text ...
3,Message-ID: <1ppvai$l79@bilbo.suite.com>\n Rep...,Entertainment,message bilbo suite reply miller suite nntp po...
4,\n Some sick part of me really liked that p...,Entertainment,sick part really like phrase actually merely t...


In [16]:
#list(set(data["filtered_text"][0].split(" ")))

#word_list = [text.split(" ") for text in data["filtered_text"]]

word_list = []
for text in data_LDA["filtered_text"]:
    temp = text.split(" ")
    word_list.append(temp)
    
    # for word in temp:
  #     if word not in word_list:
   #        word_list.append(word)
    
print(word_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import gensim.corpora as corpora

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
type(word_list[0])

list

In [19]:
# Create Dictionary 
id2word = corpora.Dictionary(word_list)

#print(id2word)

#dictionary = corpora.Dictionary([tokens])
#  print(dictionary)
    
# Create Corpus
texts = word_list

# Term Document Frequency
#corpus = id2word.doc2bow([word_list])
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
###training LDA model

from pprint import pprint
# number of topics
num_topics = 4
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 5 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"people" + 0.006*"say" + 0.006*"israel" + 0.005*"would" + '
  '0.005*"write" + 0.005*"article" + 0.005*"armenian" + 0.004*"right" + '
  '0.004*"turkish" + 0.004*"know"'),
 (1,
  '0.007*"write" + 0.005*"article" + 0.005*"say" + 0.005*"would" + '
  '0.005*"right" + 0.004*"people" + 0.004*"think" + 0.004*"state" + '
  '0.004*"know" + 0.004*"time"'),
 (2,
  '0.008*"would" + 0.007*"people" + 0.007*"write" + 0.006*"make" + '
  '0.005*"article" + 0.005*"think" + 0.004*"like" + 0.004*"say" + '
  '0.004*"israel" + 0.004*"know"'),
 (3,
  '0.010*"write" + 0.007*"would" + 0.005*"article" + 0.005*"know" + '
  '0.004*"make" + 0.004*"people" + 0.004*"think" + 0.003*"could" + '
  '0.003*"time" + 0.003*"say"')]


In [ ]:
!pip install pyLDAvis==2.1.2

In [ ]:
###Applying NNs to individual classes and for that first we create seperate data frames for each class
#import pandas as pd
#df = pd.read_csv('file_location\filename.txt', delimiter = "\t")

In [21]:
files = {}
unique = []
for name in names:
    path = base + name+'/'
    x = []
    with os.scandir(path) as entries:
        for entry in entries:
            if(entry.is_file()):
                x.append(entry.name)
    files[name] = x
    files[name].sort()

In [22]:
for k, v in files.items():
    print(k, len(v))

Entertainment 1053
Crime 1100
Science 4000
Politics 3001


In [23]:
data = {}
i = 0

for genre in files.keys() :
    texts = files[genre]
    for text in texts:
        if text in files[genre]:
            path = base + genre + '/' + text
            with open(path, "r", encoding = "latin1") as file:
                data[i] = file.readlines()
                i = i+1
            data[i-1] = [" ".join(data[i-1]), genre] 
#data
data = pd.DataFrame(data).T
print(data.shape)
data.columns = ['Text', 'Class']
data

(9154, 2)


,Text,Class
0,"\n Archive-name: ripem/faq\n Last-update: Sun,...",Entertainment
1,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment
2,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment
3,Message-ID: <1ppvai$l79@bilbo.suite.com>\n Rep...,Entertainment
4,\n Some sick part of me really liked that p...,Entertainment
...,...,...
9149,\n In article <1th4mg$53f@usenet.INS.CWRU.Edu>...,Politics
9150,\n In article <1993May24.165636.29928@cirrus.c...,Politics
9151,Distribution: world\n Message-ID: <OAF.93May25...,Politics
9152,Message-ID: <OAF.93May25095940@klosters.ai.mit...,Politics


In [24]:
data["Class"].unique()

array(['Entertainment', 'Crime', 'Science', 'Politics'], dtype=object)

In [ ]:
### Develop four seperate dataframes
#crime_df=data.loc[data['Class']=="Crime"]
#politics_df=data.loc[data['Class']=="Politics"]
#science_df=data.loc[data['Class']=="Science"]
#entertainment_df=data.loc[data['Class']=="Entertainment"]

In [25]:
wordnet_lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

for punct in punctuation:
    stop.append(punct)

def filter_text(text, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(text.lower())
    filtered_text = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha() and len(w) > 3]
    filtered_text = [wordnet_lemmatizer.lemmatize(w, pos="v") for w in filtered_text if not w in stop_words] 
    return " ".join(filtered_text)

In [26]:
data["filtered_text"] = data.Text.apply(lambda x : filter_text(x, stop)) 
data.head()

,Text,Class,filtered_text
0,"\n Archive-name: ripem/faq\n Last-update: Sun,...",Entertainment,archive name ripem last update post still rath...
1,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment,approve news answer request content type text ...
2,Approved: news-answers-request@MIT.EDU\n Conte...,Entertainment,approve news answer request content type text ...
3,Message-ID: <1ppvai$l79@bilbo.suite.com>\n Rep...,Entertainment,message bilbo suite reply miller suite nntp po...
4,\n Some sick part of me really liked that p...,Entertainment,sick part really like phrase actually merely t...


In [27]:
#list(set(data["filtered_text"][0].split(" ")))

#word_list = [text.split(" ") for text in data["filtered_text"]]

word_list = []
for text in data["filtered_text"]:
    temp = text.split(" ")
    word_list.append(temp)
    
    # for word in temp:
  #     if word not in word_list:
   #        word_list.append(word)
    
print(word_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
###### USE ONLY WHEN CLEANING IS DONE BEFORE TRAIN AND TEST SPLIT

#import gensim
#from gensim.utils import simple_preprocess
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords

#import gensim.corpora as corpora

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
###### USE ONLY WHEN CLEANING IS DONE BEFORE TRAIN AND TEST SPLIT


## Create Dictionary 
#id2word = corpora.Dictionary(word_list)


    
# # Create Corpus
#texts = word_list

# ###3Term Document Frequency

#corpus = [id2word.doc2bow(text) for text in texts]

#print(corpus)

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 2), (9, 2), (10, 2), (11, 7), (12, 2), (13, 2), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 2), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 3), (31, 2), (32, 3), (33, 4), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 2), (48, 5), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 3), (58, 3), (59, 3), (60, 1), (61, 1), (62, 1), (63, 3), (64, 3), (65, 1), (66, 1), (67, 4), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 3), (76, 1), (77, 1), (78, 4), (79, 3), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 3), (92, 2), (93, 4), (94, 2), (95, 5), (96, 4), (97, 1), (98, 1), (99, 1), (100, 1), (101, 5), (102, 1), (103, 2), (104, 2), (105, 1), (106, 1), (107, 1), (108, 2), (109, 2), (110, 1)

In [28]:
data["Class"].values

array(['Entertainment', 'Entertainment', 'Entertainment', ..., 'Politics',
       'Politics', 'Politics'], dtype=object)

In [29]:


enc_df=pd.get_dummies(data,columns=["Class"])
#enc_df
labels=enc_df[["Class_Crime","Class_Politics","Class_Science","Class_Entertainment"]]
labels

,Class_Crime,Class_Politics,Class_Science,Class_Entertainment
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
9149,0,1,0,0
9150,0,1,0,0
9151,0,1,0,0
9152,0,1,0,0


In [30]:
####dividing into input and output
X=list(data["filtered_text"])
y=labels.values

In [31]:
###### DO WE REALLY HAVE TO DO IT AS they are doing bag of words after splitting into test and train

###divide our data into training and test sets:  
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [32]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [34]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('sample_data/Data/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [35]:
###### Multi-Class Neural Network#################

deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(4, activation='softmax')(LSTM_Layer_1) #add more LSTM layer with pooling layers (1D), changing layer types,
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [36]:
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 100)          4040400   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 4,158,164
Trainable params: 117,764
Non-trainable params: 4,040,400
_________________________________________________________________
None


In [38]:
#from keras.utils import plot_model
#plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [39]:
##### train our model for 10 epochs.
TrainedModel = model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)


Epoch 1/10
46/46 [==============================] - 50s 688ms/step - loss: 0.6042 - acc: 0.3529 - val_loss: 0.5026 - val_acc: 0.4805
Epoch 2/10
46/46 [==============================] - 31s 670ms/step - loss: 0.4955 - acc: 0.4950 - val_loss: 0.4602 - val_acc: 0.5597
Epoch 3/10
46/46 [==============================] - 31s 669ms/step - loss: 0.5058 - acc: 0.4638 - val_loss: 0.5108 - val_acc: 0.4430
Epoch 4/10
46/46 [==============================] - 31s 676ms/step - loss: 0.5058 - acc: 0.4613 - val_loss: 0.5039 - val_acc: 0.4765
Epoch 5/10
46/46 [==============================] - 31s 676ms/step - loss: 0.5005 - acc: 0.4818 - val_loss: 0.4674 - val_acc: 0.5638
Epoch 6/10
46/46 [==============================] - 31s 676ms/step - loss: 0.4814 - acc: 0.5585 - val_loss: 0.4501 - val_acc: 0.6068
Epoch 7/10
46/46 [==============================] - 31s 676ms/step - loss: 0.4567 - acc: 0.5825 - val_loss: 0.4602 - val_acc: 0.5679
Epoch 8/10
46/46 [==============================] - 31s 671ms/step - 

In [40]:
### Evaluate the model
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])


58/58 [==============================] - 4s 68ms/step - loss: 0.4461 - acc: 0.5877
Test Score: 0.4460892379283905
Test Accuracy: 0.5876570343971252
